# Imports

In [1]:
import pandas as pd
import os
import numpy as np
from IPython.display import display

# Data Loading & Preprocessing

In [10]:
movies_path = '../data/movie_data.csv'
movies = pd.read_csv(movies_path, lineterminator='\n', index_col='_id')

ratings_path = '../data/ratings_export.csv'
ratings = pd.read_csv(ratings_path, lineterminator='\n', )

# users_path = '../data/users_export.csv'
# users = pd.read_csv(users_path, lineterminator='\n', )

In [11]:
movies = movies.replace({r'\r': ''}, regex=True)
movies.at['60887d7c28f29d0115ea6207', 'movie_id'] = 'nan_id'
movies.at['5fd08f22b89a841f5b234f50', 'movie_id'] = 'null_id'
try:
    movies = movies.drop(columns=['image_url', 'imdb_link', 'tmdb_id', 'tmdb_link'])
except KeyError:
    pass
movies.genres = movies.genres.replace({'[]': np.nan})

ratings = ratings.replace({'movie_id': {np.nan: 'null_id', 'nan': 'nan_id'} })
ratings = ratings.drop(ratings[ratings.user_id == 'jacksonmaines'].index)
ratings = ratings.drop(ratings[~ratings.movie_id.isin(movies.movie_id)].index)

# EDA

## Users

In [ ]:
"""
полностью бесполезная таблица
число отзывов не совпадает с реальным (из таблицы ratings)
"""

In [8]:
users

,_id,display_name,num_ratings_pages,num_reviews,username
0,5fc4172ec6cd28ebd99dd0e2,Lucy,32.0,1650,deathproof
1,5fc4172ec6cd28ebd99dd0ea,Matt Singer,52.0,1915,superpulse
2,5fc4172ec6cd28ebd99dd0ed,Sean Baker,21.0,1283,lilfilm
3,5fc4172ec6cd28ebd99dd0ee,iana,37.0,1177,ianamurray
4,5fc419171ebf67b9fbe48615,Lizzy,57.0,1810,punchdrunklizzy
...,...,...,...,...,...
8134,6239b430a936b956009c993c,Máté Turi-Mészáros,NaN,237,mateazember
8135,6239cbf7a936b95600a2d24c,Finn,NaN,297,finnvor1
8136,6239d254a936b95600a4472f,Daniel,NaN,1231,daniel
8137,6239df37a936b95600aa7aa5,ButterflyFilms,NaN,800,butterflyfilms


In [16]:
ratings[ratings.user_id=='deathproof']

,_id,movie_id,rating_val,user_id
0,5fc57c5d6758f6963451a07f,feast-2014,7,deathproof
1,5fc57c5d6758f6963451a063,loving-2016,7,deathproof
2,5fc57c5d6758f6963451a0ef,scripted-content,7,deathproof
3,5fc57c5d6758f6963451a060,the-future,4,deathproof
4,5fc57c5c6758f69634519398,mank,5,deathproof
...,...,...,...,...
10956860,623952b5a936b9560082211c,hairspray-2007,7,deathproof
10956861,623952b5a936b95600822120,bicycle-thieves,7,deathproof
10956862,623952b5a936b95600822124,the-last-days-of-disco,5,deathproof
10956863,623952b5a936b95600822126,the-novice-2021,8,deathproof


In [11]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8139 entries, 0 to 8138
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   _id                8139 non-null   object 
 1   display_name       7832 non-null   object 
 2   num_ratings_pages  4747 non-null   float64
 3   num_reviews        8139 non-null   int64  
 4   username           8139 non-null   object 
dtypes: float64(1), int64(1), object(3)
memory usage: 318.1+ KB


## Ratings

In [4]:
# просто оценки, потом смёрджим с фильмами

In [9]:
ratings

,_id,movie_id,rating_val,user_id
0,5fc57c5d6758f6963451a07f,feast-2014,7,deathproof
1,5fc57c5d6758f6963451a063,loving-2016,7,deathproof
2,5fc57c5d6758f6963451a0ef,scripted-content,7,deathproof
3,5fc57c5d6758f6963451a060,the-future,4,deathproof
4,5fc57c5c6758f69634519398,mank,5,deathproof
...,...,...,...,...
11078162,6239f4f1a936b95600b3d798,alien-3,6,turnitip
11078163,6239f4f1a936b95600b3d799,battleship-potemkin,7,turnitip
11078164,6239f4f1a936b95600b3d79e,pusher,6,turnitip
11078165,6239f4f1a936b95600b3d7a1,wild-strawberries,7,turnitip


In [7]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11075887 entries, 0 to 11078166
Data columns (total 4 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   _id         object
 1   movie_id    object
 2   rating_val  int64 
 3   user_id     object
dtypes: int64(1), object(3)
memory usage: 422.5+ MB


## Movies

In [8]:
movies

,genres,imdb_id,movie_id,movie_title,original_language,overview,popularity,production_countries,release_date,runtime,spoken_languages,vote_average,vote_count,year_released
_id,,,,,,,,,,,,,,
5fc85f606758f69634496fd3,"[""Music"",""Animation""]",NaN,football-freaks,Football Freaks,en,"Football crazy, football mad. Don’t watch this...",0.600,"[""United Kingdom""]",1971-12-05,0.0,[],0.0,0.0,1971.0
5fc85ff26758f696344ace0c,NaN,tt0586129,aftermath-1960,Aftermath,en,Aftermath was the pilot for an unsold TV serie...,0.600,[],1960-04-17,22.0,[],8.0,1.0,1960.0
5fc85f606758f69634496fcd,"[""Drama""]",tt0045731,where-chimneys-are-seen,Where Chimneys Are Seen,ja,Gosho’s most celebrated film both in Japan and...,1.568,"[""Japan""]",1953-03-05,108.0,"[""日本語""]",6.6,10.0,1953.0
5fc85f606758f69634496fd1,"[""Drama""]",tt0187327,the-musicians-daughter,The Musician's Daughter,en,Carl Wagner's good wife was dying. His heart b...,0.600,"[""United States of America""]",1911-12-12,15.0,[],0.0,0.0,1911.0
5fc85f606758f69634496fd4,"[""Documentary""]",tt4769914,50-years-of-fabulous,50 Years of Fabulous,en,50 Years of Fabulous recounts the rich history...,0.600,[],2018-05-17,75.0,[],0.0,0.0,2018.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6239f32da936b95600b301d8,NaN,NaN,the-western-world-of-sailor-moon,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6239f32da936b95600b301e8,NaN,NaN,shadowplay,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6239f32da936b95600b301f2,NaN,NaN,only-always-you,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
Index: 285504 entries, 5fc85f606758f69634496fd3 to 6239f32da936b95600b30204
Data columns (total 14 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   genres                222526 non-null  object 
 1   imdb_id               243802 non-null  object 
 2   movie_id              285504 non-null  object 
 3   movie_title           283340 non-null  object 
 4   original_language     274872 non-null  object 
 5   overview              261248 non-null  object 
 6   popularity            274872 non-null  float64
 7   production_countries  274872 non-null  object 
 8   release_date          271050 non-null  object 
 9   runtime               270055 non-null  float64
 10  spoken_languages      274872 non-null  object 
 11  vote_average          274872 non-null  float64
 12  vote_count            274872 non-null  float64
 13  year_released         277932 non-null  float64
dtypes: float64(5), o

In [ ]:
movies.production_countries.dropna()

_id
5fc85f606758f69634496fd3              ["United Kingdom"]
5fc85ff26758f696344ace0c                              []
5fc85f606758f69634496fcd                       ["Japan"]
5fc85f606758f69634496fd1    ["United States of America"]
5fc85f606758f69634496fd4                              []
                                        ...             
6239e60ea936b95600aeb9bb                              []
6239e60ea936b95600aeb9cf                       ["India"]
6239e60fa936b95600aeb9e4    ["United States of America"]
6239e60fa936b95600aeb9e5                              []
6239e61ba936b95600aed5d3                              []
Name: production_countries, Length: 274872, dtype: object

In [ ]:
ratings[ratings.movie_id=='shadowplay']

,_id,movie_id,rating_val,user_id
11077902,6239f32ca936b95600b2f351,shadowplay,4,nyavaeh


In [19]:
movies[movies.overview.isna()].info()

<class 'pandas.core.frame.DataFrame'>
Index: 24256 entries, 5fc85ff26758f696344ace53 to 6239f32da936b95600b30204
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   genres                8520 non-null   object 
 1   imdb_id               15831 non-null  object 
 2   movie_id              24256 non-null  object 
 3   movie_title           22092 non-null  object 
 4   original_language     13624 non-null  object 
 5   overview              0 non-null      object 
 6   popularity            13624 non-null  float64
 7   production_countries  13624 non-null  object 
 8   release_date          13207 non-null  object 
 9   runtime               12090 non-null  float64
 10  spoken_languages      13624 non-null  object 
 11  vote_average          13624 non-null  float64
 12  vote_count            13624 non-null  float64
 13  year_released         20130 non-null  float64
dtypes: float64(5), object(9)
memory u

In [20]:
only_movie_id = movies[movies.overview.isna() & movies.movie_title.isna()]
only_movie_id.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2164 entries, 5fc85ff26758f696344aceff to 6239f32da936b95600b30204
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   genres                0 non-null      object 
 1   imdb_id               0 non-null      object 
 2   movie_id              2164 non-null   object 
 3   movie_title           0 non-null      object 
 4   original_language     0 non-null      object 
 5   overview              0 non-null      object 
 6   popularity            0 non-null      float64
 7   production_countries  0 non-null      object 
 8   release_date          0 non-null      object 
 9   runtime               0 non-null      float64
 10  spoken_languages      0 non-null      object 
 11  vote_average          0 non-null      float64
 12  vote_count            0 non-null      float64
 13  year_released         0 non-null      float64
dtypes: float64(5), object(9)
memory us

In [26]:
ratings[ratings.movie_id.isin(only_movie_id.movie_id)].groupby('movie_id').count().sort_values('_id')

,_id,rating_val,user_id
movie_id,,,
1-across,1,1,1
sharp-shadows-grey-skies-and-downers,1,1,1
wwe-money-in-the-bank-2010,1,1,1
gods-of-war-assembling-earths-mightiest-anti-heroes,1,1,1
godzilla-battle-royale-2014,1,1,1
...,...,...,...
mystery-of-the-seeing-hand-and-golden-sphere,83,83,83
to-boldly-flee,83,83,83
tdf-really-works,110,110,110


In [16]:
movies.columns

Index(['genres', 'imdb_id', 'movie_id', 'movie_title', 'original_language',
       'overview', 'popularity', 'production_countries', 'release_date',
       'runtime', 'spoken_languages', 'vote_average', 'vote_count',
       'year_released'],
      dtype='object')